In [55]:
import sys
sys.path.append('../')

from handles.marketdata import *

# Index Handle - oggetti statici

In [56]:
from index.curverateindex import IborIndex
from instruments.swap import SwapFixedFloating

class MarketIndex:
    # da fare config file con mappaggio curva-indice
    eur_1m = IborIndex('EUR-EURIBOR-1M',
                        TARGET(),
                        1
                        )
    
    eur_3m = IborIndex('EUR-EURIBOR-3M',
                        TARGET(),
                        3
                        )
    
    eur_6m = IborIndex('EUR-EURIBOR-6M',
                        TARGET(),
                        6
                        )
    
    eur_12m = IborIndex('EUR-EURIBOR-12M',
                        TARGET(),
                        12
                        )
   

# Load Portfolio

In [61]:
import json

class DataModel:
    with open('../data/trades/TradesAggregated_2.json', 'r') as file:
        trade_data = json.load(file)
        
def make_swap(trade):
    ''' 
    Read json and returns tq.Swap
    '''
    leg1_schedule_len = len(trade['data']['Leg1.schedule']['rows'])
    leg2_schedule_len = len(trade['data']['Leg2.schedule']['rows'])

    leg1_is_fixed = trade['data']['Leg1.isFixedLeg']
    leg2_is_fixed = trade['data']['Leg2.isFixedLeg']

    if not leg1_is_fixed:
        leg1_index = trade['data']['Leg1.index']
    elif not leg2_is_fixed:
        leg2_index = trade['data']['Leg2.index']

    date_format = '%Y-%m-%d'
    leg1_payment_dates = [trade['data']['Leg1.schedule']['rows'][i]['paymentDate'] for i in range(leg1_schedule_len)]
    leg1_payment_dates = [datetime.strptime(dt, date_format).date() for dt in leg1_payment_dates]
    leg2_payment_dates = [trade['data']['Leg2.schedule']['rows'][i]['paymentDate'] for i in range(leg2_schedule_len)]
    leg2_payment_dates = [datetime.strptime(dt, date_format).date() for dt in leg2_payment_dates]

    leg1_notionals = [trade['data']['Leg1.notionalTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_notionals = [trade['data']['Leg2.notionalTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_margins = [trade['data']['Leg1.marginTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_margins = [trade['data']['Leg2.marginTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_rate_factor = [trade['data']['Leg1.rateFactorTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_rate_factor = [trade['data']['Leg2.rateFactorTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_fixed_payments = [trade['data']['Leg1.fixedPaymentAmountTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_fixed_payments = [trade['data']['Leg2.fixedPaymentAmountTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    if leg1_is_fixed and not leg2_is_fixed:

        if leg2_index == 'EUR-EURIBOR-1M':
            index = MarketIndex.eur_1m
        elif leg2_index == 'EUR-EURIBOR-3M':
            index = MarketIndex.eur_3m
        elif leg2_index == 'EUR-EURIBOR-6M':
            index = MarketIndex.eur_6m
        elif leg2_index == 'EUR-EURIBOR-12M':
            index = MarketIndex.eur_12m
        else:
            raise ValueError('No index available')

        swap_obj = SwapFixedFloating(leg1_payment_dates,
                                    leg2_payment_dates,
                                    leg1_notionals,
                                    leg2_notionals ,
                                    leg2_rate_factor, # moltiplicatore variabile
                                    leg2_margins, # spread variabile
                                    index,
                                    leg1_margins) # tasso fisso
        
    elif leg2_is_fixed and not leg1_is_fixed:
        if leg1_index == 'EUR-EURIBOR-6M':
            index = MarketIndex.eur_6m_index
        elif leg1_index == 'EUR-EURIBOR-3M':
            index = MarketIndex.eur_3m
        elif leg1_index == 'EUR-EURIBOR-6M':
            index = MarketIndex.eur_6m
        elif leg1_index == 'EUR-EURIBOR-12M':
            index = MarketIndex.eur_12m
        else:
            raise ValueError('No index available')

        swap_obj = SwapFixedFloating(leg1_payment_dates,
                                    leg2_payment_dates,
                                    leg1_notionals,
                                    leg2_notionals,
                                    leg1_rate_factor, # moltiplicatore variabile
                                    leg1_margins, # spread variabile
                                    index,
                                    leg2_margins) # tasso fisso    
    return swap_obj

def make_portfolio(trade_data):
    portfolio = {}
    for trade_json in trade_data:
        try:
            if trade_json['data']['productType'] == 'InterestRateSwapScript':
                obj = make_swap(trade_json)
                obj.counterparty = trade_json['data']['cpty']
                portfolio[trade_json['ref']] = obj
            # if cross currency: make_ccs()

        except Exception as error:
            portfolio[trade_json['ref']] = error
    return portfolio


In [64]:
# load portfolio into memory
ptf = make_portfolio(DataModel.trade_data)
# swap object test
swap_test = ptf['0-17092454']

# Load curve

In [68]:
mdl = MarketDataLoader() # source

# Pricing test

In [71]:
import tensorflow as tf
from tensorflow.python.framework import dtypes

In [72]:
class RateCurve:
    def __init__(self, pillars, rates):
        ''' 
        classe dummy per le curve
        '''
        self.pillars = pillars # list
        self.rates = rates     # tensor list

    def discount(self, term):
        if term <= self.pillars[0]:
            nrt = -term * self.rates[0]
            df = tf.exp(nrt)
            return df
        if term >= self.pillars[-1]:
            nrt = -term * self.rates[-1]
            df = tf.exp(nrt)
            return df
        for i in range(0, len(self.pillars) - 1):
            if term < self.pillars[i + 1]:
                dtr = 1 / (self.pillars[i + 1] - self.pillars[i])
                w1 = (self.pillars[i + 1] - term) * dtr
                w2 = (term - self.pillars[i]) * dtr
                r1 = w1 * self.rates[i]
                r2 = w2 * self.rates[i + 1]
                rm = r1 + r2
                nrt = -term * rm
                df = tf.exp(nrt)
                return df

In [73]:
# add fixing storici
MarketIndex.eur_6m.add_fixing(date(2023,6,30), 0.03)

In [74]:
curve_rates = mdl.ir_eur_curve_estr['quote'].to_list()
curve_pillars = (mdl.ir_eur_curve_estr['daycount']/365).to_list()

rc_rates_tensor = [tf.Variable(r, dtype=dtypes.float64) for r in curve_rates]
curve_tensor = RateCurve(curve_pillars, rc_rates_tensor)

In [75]:
with tf.GradientTape() as tape:
    npv = swap_test.price(curve_tensor, curve_tensor, mdl.evaluation_date)
    #sum = tf.reduce_sum(npv, 0)

In [76]:
sensitivities = tape.gradient(npv, curve_tensor.rates)